In [68]:
#important packeges
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [69]:
#load dataset
World_cup=pd.read_csv("../datasets/FIFA-2018-World-cup/World Cup 2018 Dataset.csv")
results=pd.read_csv("../datasets/FIFA-2018-World-cup/results.csv")
World_cup.head(3)

,Team,Group,Previous \nappearances,Previous \ntitles,Previous\n finals,Previous\n semifinals,Current \nFIFA rank,First match \nagainst,Match index,history with \nfirst opponent\n W-L,history with\n first opponent\n goals,Second match\n against,Match index.1,history with\n second opponent\n W-L,history with\n second opponent\n goals,Third match\n against,Match index.2,history with\n third opponent\n W-L,history with\n third opponent\n goals,Unnamed: 19
0,Russia,A,10.0,0.0,0.0,1.0,65.0,Saudi Arabia,1.0,-1.0,-2.0,Egypt,17.0,NaN,NaN,Uruguay,33.0,0.0,0.0,NaN
1,Saudi Arabia,A,4.0,0.0,0.0,0.0,63.0,Russia,1.0,1.0,2.0,Uruguay,18.0,1.0,1.0,Egypt,34.0,-5.0,-5.0,NaN
2,Egypt,A,2.0,0.0,0.0,0.0,31.0,Uruguay,2.0,-1.0,-2.0,Russia,17.0,NaN,NaN,Saudi Arabia,34.0,5.0,5.0,NaN


# Data exploration & visualization


In [70]:
#function for each feature(datatype & null )
def data_info(data):
    total_null = data.isnull().sum()
    tt=pd.DataFrame(total_null,columns=["n_null"])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    if(data.isna().sum().any())==False:
        print("data has no null values")
    else:
        print("data has null values")
    
    return(np.transpose(tt.sort_values(by="n_null",ascending=False)))

In [71]:
data_info(World_cup)

data has null values


,Unnamed: 19,history with \nfirst opponent\n W-L,history with\n first opponent\n goals,history with\n second opponent\n goals,history with\n second opponent\n W-L,history with\n third opponent\n goals,history with\n third opponent\n W-L,Previous \ntitles,Match index.1,Match index.2,Third match\n against,Second match\n against,Previous\n finals,Group,Previous \nappearances,Match index,First match \nagainst,Current \nFIFA rank,Previous\n semifinals,Team
n_null,33,13,13,11,11,5,5,1,1,1,1,1,1,1,1,1,1,1,1,1
Types,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,float64,object,float64,float64,object,float64,float64,object


In [72]:
data_info(results)

data has no null values


,date,home_team,away_team,home_score,away_score,tournament,city,country
n_null,0,0,0,0,0,0,0,0
Types,object,object,object,int64,int64,object,object,object


In [73]:
results.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
home_score,38902.0,1.727803,1.721628,0.0,1.0,1.0,2.0,31.0
away_score,38902.0,1.194129,1.397114,0.0,0.0,1.0,2.0,22.0


In [74]:
results.describe(include='O').transpose()

,count,unique,top,freq
date,38902,14786,2012-02-29,66
home_team,38902,241,Brazil,546
away_team,38902,243,Uruguay,520
tournament,38902,95,Friendly,16381
city,38902,1802,Kuala Lumpur,571
country,38902,263,USA,1084


In [75]:
#results for teams participate in world cup

In [76]:
winner=[]
for i in results.index:
    if results["home_score"][i]>results["away_score"][i]:
        winner.append(results["home_team"][i])
    elif results["home_score"][i]==results["away_score"][i]:
        winner.append("draw")
    else:
        winner.append(results["away_team"][i])
results["winner"]=winner

In [77]:
#adding goal difference col
results["goal_difference"]= np.absolute(results["home_score"]- results["away_score"])

In [78]:
results.tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,winner,goal_difference
38897,2018-03-27,Mongolia,Mauritius,0,2,Friendly,Ulan Bator,Mongolia,Mauritius,2
38898,2018-03-27,Seychelles,Swaziland,0,0,Friendly,Victoria,Seychelles,draw,0
38899,2018-03-27,Kosovo,Burkina Faso,2,0,Friendly,Franconville,France,Kosovo,2
38900,2018-03-27,Iraq,Syria,1,1,Friendly,Basra,Iraq,draw,0
38901,2018-04-01,Malaysia,Bhutan,7,0,Friendly,Kuala Lumpur,Malaysia,Malaysia,7


In [79]:
#all 32 country that will participate 
World_cup_teams= World_cup.Team.unique()
World_cup_teams

array(['Russia', 'Saudi Arabia', 'Egypt', 'Uruguay', 'Porugal', 'Spain',
       'Morocco', 'IRAN', 'France', 'Australia', 'Peru', 'Denmark',
       'Argentina', 'Iceland', 'Croatia', 'Nigeria', 'Brazil',
       'Switzerland', 'Costarica', 'Serbia', 'Germany', 'Mexico',
       'Sweden', 'Korea', 'Belgium', 'Panama', 'Tunisia', 'England',
       'Poland', 'Senegal', 'Columbia', 'Japan', nan], dtype=object)

In [80]:
World_cup_home=results[results["home_team"].isin(World_cup_teams)]
World_cup_away=results[results["away_team"].isin(World_cup_teams)]

In [81]:
df_teams=pd.concat((World_cup_home,World_cup_away))
df_teams.drop_duplicates()

,date,home_team,away_team,home_score,away_score,tournament,city,country,winner,goal_difference
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,England,2
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,draw,0
6,1877-03-03,England,Scotland,1,3,Friendly,London,England,Scotland,2
10,1879-01-18,England,Wales,2,1,Friendly,London,England,England,1
11,1879-04-05,England,Scotland,5,4,Friendly,London,England,England,1
...,...,...,...,...,...,...,...,...,...,...
38851,2018-03-26,Wales,Uruguay,0,1,Friendly,Nanning,China,Uruguay,1
38867,2018-03-27,Greece,Egypt,1,0,Friendly,Zurich,Switzerland,Greece,1
38877,2018-03-27,Ukraine,Japan,2,1,Friendly,Liège,Belgium,Ukraine,1
38880,2018-03-27,Romania,Sweden,1,0,Friendly,Craiova,Romania,Romania,1


In [82]:
year=[]
for date in df_teams["date"]:
    year.append(int(date[:4]))
df_teams["year"]=year
df_teams_1930=df_teams[df_teams["year"] >= 1930]
df_teams_1930

,date,home_team,away_team,home_score,away_score,tournament,city,country,winner,goal_difference,year
1230,1930-01-01,Spain,Czechoslovakia,1,0,Friendly,Barcelona,Spain,Spain,1,1930
1238,1930-03-02,Germany,Italy,0,2,Friendly,Frankfurt am Main,Germany,Italy,2,1930
1240,1930-03-23,France,Switzerland,3,3,Friendly,Colombes,France,draw,0,1930
1241,1930-04-05,England,Scotland,5,2,British Championship,London,England,England,3,1930
1243,1930-04-13,France,Belgium,1,6,Friendly,Colombes,France,Belgium,5,1930
...,...,...,...,...,...,...,...,...,...,...,...
38877,2018-03-27,Ukraine,Japan,2,1,Friendly,Liège,Belgium,Ukraine,1,2018
38878,2018-03-27,Belgium,Saudi Arabia,4,0,Friendly,Brussels,Belgium,Belgium,4,2018
38880,2018-03-27,Romania,Sweden,1,0,Friendly,Craiova,Romania,Romania,1,2018
38881,2018-03-27,Bosnia-Herzegovina,Senegal,0,0,Friendly,Le Havre,France,draw,0,2018


In [83]:
df_teams_1930=df_teams_1930.drop(["date","tournament","city","country","year","home_score","away_score","goal_difference"],axis=1)
df_teams_1930

,home_team,away_team,winner
1230,Spain,Czechoslovakia,Spain
1238,Germany,Italy,Italy
1240,France,Switzerland,draw
1241,England,Scotland,England
1243,France,Belgium,Belgium
...,...,...,...
38877,Ukraine,Japan,Ukraine
38878,Belgium,Saudi Arabia,Belgium
38880,Romania,Sweden,Romania
38881,Bosnia-Herzegovina,Senegal,draw


In [84]:
df_teams_1930=df_teams_1930.reset_index(drop=True)
df_teams_1930.loc[df_teams_1930.home_team == df_teams_1930.winner ,"winner"]=1
df_teams_1930.loc[df_teams_1930.away_team == df_teams_1930.winner ,"winner"]=2
df_teams_1930.loc[df_teams_1930.winner == "draw" ,"winner"]=0
df_teams_1930

,home_team,away_team,winner
0,Spain,Czechoslovakia,1
1,Germany,Italy,2
2,France,Switzerland,0
3,England,Scotland,1
4,France,Belgium,2
...,...,...,...
16963,Ukraine,Japan,1
16964,Belgium,Saudi Arabia,1
16965,Romania,Sweden,1
16966,Bosnia-Herzegovina,Senegal,0


In [85]:
final=pd.get_dummies(df_teams_1930,prefix=["home_team","away_team"],columns=["home_team","away_team"])
x=final.drop("winner",axis=1)
y=final["winner"]
y=y.astype('int')
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [86]:
y_train

6858     2
11538    2
9466     2
11851    2
13768    1
        ..
3922     2
5598     1
11973    2
12577    0
8815     2
Name: winner, Length: 11877, dtype: int32

In [87]:
logreg=LogisticRegression()
logreg.fit(x_train,y_train.values.ravel())
print(logreg.score(x_train,y_train))
print(logreg.score(x_test,y_test))

C:\Users\abanop\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.573629704470826
0.5572579061088195


In [88]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train, y_train.values.ravel())
Y_pred = decision_tree.predict(x_test)
acc_decision_tree = round(decision_tree.score(x_train, y_train) * 100, 2)
acc_decision_tree

67.31

In [89]:
fifa_rankings=pd.read_csv("../datasets/FIFA-2018-World-cup/fifa_rankings.csv")
fixtures=pd.read_csv("../datasets/FIFA-2018-World-cup/fixtures.csv")
pred_set=[]

In [90]:
fixtures.insert(1,"home_ranking",fixtures["Home Team"].map(fifa_rankings.set_index("Team")["Position"]))
fixtures.insert(2,"away_ranking",fixtures["Away Team"].map(fifa_rankings.set_index("Team")["Position"]))

In [91]:
fixtures

,Round Number,home_ranking,away_ranking,Date,Location,Home Team,Away Team,Group,Result
0,1,66.0,70.0,14/06/2018 18:00,"Luzhniki Stadium, Moscow",Russia,Saudi Arabia,Group A,NaN
1,1,46.0,17.0,15/06/2018 15:00,Ekaterinburg Stadium,Egypt,Uruguay,Group A,NaN
2,1,42.0,36.0,15/06/2018 18:00,Saint Petersburg Stadium,Morocco,Iran,Group B,NaN
3,1,4.0,8.0,15/06/2018 21:00,"Fisht Stadium, Sochi",Portugal,Spain,Group B,NaN
4,1,7.0,40.0,16/06/2018 13:00,Kazan Arena,France,Australia,Group C,NaN
...,...,...,...,...,...,...,...,...,...
59,Quarter Finals,NaN,NaN,07/07/2018 21:00,"Fisht Stadium, Sochi",To be announced,To be announced,NaN,NaN
60,Semi Finals,NaN,NaN,10/07/2018 21:00,Saint Petersburg Stadium,To be announced,To be announced,NaN,NaN
61,Semi Finals,NaN,NaN,11/07/2018 21:00,"Luzhniki Stadium, Moscow",To be announced,To be announced,NaN,NaN
62,Finals,NaN,NaN,14/07/2018 17:00,Saint Petersburg Stadium,To be announced,To be announced,NaN,NaN


In [92]:
for index,row in fixtures.iterrows():
    if row["home_ranking"]>row["away_ranking"]:
        pred_set.append({"Home Team":row["Home Team"],"Away Team":row["Away Team"],"winner":None})
    else:
        pred_set.append({"Home Team":row["Away Team"],"Away Team":row["Home Team"],"winner":None})
pred_set_df=pd.DataFrame(pred_set)      
#used later
backup_pred_set=pred_set_df

In [93]:
pred_set_df=pd.get_dummies(pred_set_df,prefix=["Home Team","Away Team"],columns=["Home Team","Away Team"])

In [98]:
final.shape

(16968, 407)

In [94]:
#teams not participate in world cup
missing_cols=set(final.columns) -set(pred_set_df.columns) 
for c in missing_cols:
    pred_set_df[c]=0
pred_set_df= pred_set_df[final.columns]
#pred_set_df

In [99]:
pred_set_df=pred_set_df.drop("winner",axis=1)

In [100]:
#group matches 
predictions = logreg.predict(pred_set_df)

In [103]:
for i in range(fixtures.shape[0]):
    print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
    if predictions[i] == 2:
        print("Winner: " + backup_pred_set.iloc[i, 1])
    elif predictions[i] == 1:
        print("Draw")
    elif predictions[i] == 0:
        print("Winner: " + backup_pred_set.iloc[i, 0])
'''' print('Probability of ' + backup_pred_set.iloc[i, 1] + ' winning: ', '%.3f'%(logreg.predict_proba(pred_set_df)[i][2]))
    print('Probability of Draw: ', '%.3f'%(logreg.predict_proba(pred_set)[i][1]))
    print('Probability of ' + backup_pred_set.iloc[i, 0] + ' winning: ', '%.3f'%(logreg.predict_proba(pred_set_df)[i][0]))
    print("") '''

Russia and Saudi Arabia
Draw
Uruguay and Egypt
Draw
Iran and Morocco
Draw
Portugal and Spain
Draw
France and Australia
Draw
Argentina and Iceland
Draw
Peru and Denmark
Draw
Croatia and Nigeria
Draw
Costa Rica and Serbia
Draw
Germany and Mexico
Draw
Brazil and Switzerland
Draw
Sweden and Korea Republic
Draw
Belgium and Panama
Draw
England and Tunisia
Draw
Colombia and Japan
Draw
Poland and Senegal
Draw
Egypt and Russia
Draw
Portugal and Morocco
Draw
Uruguay and Saudi Arabia
Draw
Spain and Iran
Draw
Denmark and Australia
Draw
France and Peru
Draw
Argentina and Croatia
Draw
Brazil and Costa Rica
Draw
Iceland and Nigeria
Draw
Switzerland and Serbia
Draw
Belgium and Tunisia
Draw
Mexico and Korea Republic
Draw
Germany and Sweden
Draw
England and Panama
Draw
Senegal and Japan
Draw
Poland and Colombia
Draw
Uruguay and Russia
Draw
Egypt and Saudi Arabia
Draw
Portugal and Iran
Draw
Spain and Morocco
Draw
France and Denmark
Draw
Peru and Australia
Draw
Argentina and Nigeria
Draw
Croatia and Icela

'\' print(\'Probability of \' + backup_pred_set.iloc[i, 1] + \' winning: \', \'%.3f\'%(logreg.predict_proba(pred_set_df)[i][2]))\n    print(\'Probability of Draw: \', \'%.3f\'%(logreg.predict_proba(pred_set)[i][1]))\n    print(\'Probability of \' + backup_pred_set.iloc[i, 0] + \' winning: \', \'%.3f\'%(logreg.predict_proba(pred_set_df)[i][0]))\n    print("") '